In [1]:
## basic functionality
import pandas as pd
import numpy as np
import re
import os
#fuzzywuzzy packages 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import Levenshtein
#parallel processing 
import joblib
from joblib import Parallel, delayed
import geopandas as gpd
## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from geopandas import GeoDataFrame
import datetime
import time
import random 
import matplotlib.pyplot as plt 
import seaborn as sns
import janitor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


## useful material for fuzz match 
#https://stackoverflow.com/questions/31806695/when-to-use-which-fuzz-function-to-compare-2-strings

In [2]:
df_cen = pd.read_csv("../data/census_infra_od.csv")

C:\Users\Lahiri\AppData\Local\Temp\ipykernel_51020\1099682092.py:1: DtypeWarning: Columns (235,286,304,312,370,371,373,374,376,377,393) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cen = pd.read_csv("../data/census_infra_od.csv")


In [3]:
df_cen.shape
#exp.shape

(51313, 396)

### Creating groups of variables
Group columns by type, manually sorted

In [4]:
road_cols = [col for col in df_cen.columns if ("Road" in col or 'Highway' in col)] 

fin_cols = ['ATM (Status A(1)/NA(2))',
            'Commercial Bank (Status A(1)/NA(2))',
            'Cooperative Bank (Status A(1)/NA(2))',
            'Agricultural Credit Societies (Status A(1)/NA(2))',
            'Self - Help Group (SHG) (Status A(1)/NA(2))']

power_cols = ['Power Supply For All Users Summer (April-Sept.) per day (in Hours)',
              'Power Supply For All Users Winter (Oct.-March) per day (in Hours)']

health_cols = ['Community Health Centre (Numbers)',
               'Community Health Centre Doctors Total Strength (Numbers)',
               'Community Health Centre Doctors In Position (Numbers)',
               'Community Health Centre Para Medical  Staff Total Strength (Numbers)',
               'Community Health Centre Para Medical Staff In Position (Numbers)',
               'Primary Health Centre (Numbers)',
               'Primary Health Centre  Doctors Total Strength (Numbers)', 
               'Primary Health  Centre Doctors In Position (Numbers)',
               'Primary  Health Centre Para Medical  Staff Total Strength (Numbers)',
               'Primary  Health Centre Para Medical Staff In Position (Numbers)',
               'Primary Heallth Sub Centre (Numbers)',
               'Primary Heallth Sub Centre Doctors Total Strength (Numbers)',
               'Primary Heallth Sub Centre Doctors In Position (Numbers)',
               'Primary Heallth Sub Centre Para Medical  Staff Total Strength (Numbers)',
               'Primary Heallth Sub Centre Para Medical Staff In Position (Numbers)',
               'Maternity And Child Welfare Centre (Numbers)',
               'Maternity And Child Welfare Centre Doctors Total Strength (Numbers)',
               'Maternity And Child Welfare Centre Doctors In Position (Numbers)',
               'Maternity And Child Welfare Centre Para Medical  Staff Total Strength (Numbers)',
               'Maternity And Child Welfare Centre Para Medical Staff In Position (Numbers)',
               'TB Clinic (Numbers)', 'TB Clinic Doctors Total Strength (Numbers)',
               'TB Clinic Doctors In Position (Numbers)',
               'TB Clinic Para Medical Para Medical  Staff Total Strength (Numbers)',
               'TB Clinic Para Medical Para Medical Staff In Position (Numbers)',
               'Hospital Allopathic (Numbers)', 
               'Hospital Allopathic Doctors Total Strength (Numbers)', 
               'Hospital Allopathic Doctors In Position (Numbers)',
               'Hospital Allopathic Para Medical  Staff Total Strength (Numbers)',
               'Hospital Allopathic Para Medical Staff In Position (Numbers)',
               'Hospiltal Alternative Medicine (Numbers)',
               'Hospiltal Alternative Medicine Doctors Total Strength (Numbers)',
               'Hospiltal Alternative Medicine Doctors In Position (Numbers)',
               'Hospiltal Alternative Medicine Para Medical  Staff Total Strength (Numbers)',
               'Hospiltal Alternative Medicine Para Medical Staff In Position (Numbers)',
               'Dispensary (Numbers)',
               'Dispensary Doctors Total Strength (Numbers)',
               'Dispensary Doctors In Position (Numbers)',
               'Dispensary Para Medical  Staff Total Strength (Numbers)',
               'Dispensary Para Medical Staff In Position (Numbers)',
               'Veterinary Hospital (Numbers)',
               'Veterinary Hospital Doctors Total Strength (Numbers)',
               'Veterinary Hospital Doctors In Position (Numbers)',
               'Veterinary Hospital Para Medical  Staff Total Strength (Numbers)',
               'Veterinary Hospital Para Medical Staff In Position (Numbers)',
               'Mobile Health Clinic (Numbers)', 
               'Mobile Health Clinic Doctors Total Strength (Numbers)', 
               'Mobile Health Clinic Doctors In Position (Numbers)',
               'Mobile Health Clinic Para Medical  Staff Total Strength (Numbers)',
               'Mobile Health Clinic Para Medical Staff In Position(Numbers)', 
               'Family Welfare Centre (Numbers)', 
               'Family Welfare Centre Doctors Total Strength (Numbers)',
               'Family Welfare Centre Doctors In Position (Numbers)',
               'Family Welfare Centre Para Medical  Staff Total Strength (Numbers)',
               'Family Welfare Centre Para Medical Staff In Position (Numbers)',
               'Non Government Medical facilities Out Patient (Numbers)',
               'Non Government Medical facilities In And Out Patient (Numbers)',
               'Non Government Medical facilities Charitable (Numbers)',
               'Non Government Medical facilities Medical Prctitioner with MBBS Degree (Numbers) ',
               'Non Government Medical facilities Medical Prctitioner with other Degree (Numbers)  ', 
               'Non Government Medical facilities Medical Practitioner with no  Degree (Numbers) ',
               'Non Government Medical facilities Traditional Practitioner and Faith  Healer (Numbers) ', 
               'Non Government Medical facilities Medicine Shop (Numbers) ',
               'Non Government Medical facilities Others (Numbers)']

educ_cols = ['Govt  Pre - Primary School (Nursery/LKG/UKG) (Numbers)',
             'Private Pre - Primary School (Nursery/LKG/UKG) (Numbers)', 
             'Govt Primary School (Numbers)', 
             'Private  Primary School (Numbers)',
             'Govt  Middle School (Numbers)',
             'Private Middle School (Numbers)', 
             'Govt  Secondary School (Numbers)',
             'Private Secondary  School (Numbers)', 
             'Govt Senior Secondary School (Numbers)',
             'Private Senior Secondary  School (Numbers)',
             'Govt  Arts and Science Degree College (Numbers)',
             'Private  Arts and Science Degree College (Numbers)', 
             'Govt Engineering College (Numbers)',
             'Private Engineering College (Numbers)',
             'Govt Medicine  College (Numbers)', 
             'Private Medicine College (Numbers)', 
             'Govt Management Institute (Numbers)',
             'Private Management Institute (Numbers)', 
             'Govt Polytechnic (Numbers)', 
             'Private Polytechnic (Numbers)', 
             'Govt Vocational Training School/ITI (Numbers)',
             'Private Vocational Training School/ITI (Numbers)', 
             'Government Non Formal Training Centre (Numbers)',
             'Private Non Formal Training Centre (Numbers)', 
             'Government School For Disabled (Numbers)', 
             'Private School For Disabled (Numbers)', 
             'Government Others (Numbers)', 
             'Private Others (Numbers)']

### Recoding Binary

Variables with 'Status' in column name are coded as 1 = Available, 2=Unavalaibale.

Since we will add the variables, we need binary coding (1/0).

In [5]:
# Some binary, others will be numeric

# For atm, bank variables

for col in df_cen[fin_cols]:
    df_cen[col].replace(2,0, inplace=True)

#For road vars
for col in df_cen[road_cols]:
    df_cen[col].replace(2,0, inplace=True)

ADD INDICATORS TO GENERATE SCORES

### Sum columns for amenities

two kinds of columns, numerical (continuous and binary-recoded above)

In [6]:
# df_cen['fin_sum']=df_cen[fin_cols].sum(axis=1)
# df_cen['roads_sum'] = df_cen[road_cols].sum(axis=1)
# df_cen['power_hrs_sum'] = df_cen[power_cols].sum(axis=1)

In [7]:
df_cen['dist_cen'] = df_cen['District Name'].str.lower().str.strip()
df_cen['blk_cen'] = df_cen['CD Block Name'].str.lower().str.strip()
df_cen['gp_cen'] =df_cen['Gram Panchayat Name'].str.lower().str.strip()
df_cen['vill_cen'] = df_cen['Village Name'].str.lower().str.strip()



#### Changing district names 

Four district names were different between two variables. Rather than depending on `fuzz` match, changing them manually would yield better results during fuzz match 

In [8]:
conditions = [df_cen.dist_cen == 'baudh',df_cen.dist_cen == 'debagarh',df_cen.dist_cen == 'nabarangapur',df_cen.dist_cen == 'subarnapur']
change_with = ['boudh', 'deogarh', 'nabarangpur', 'sonepur']
df_cen['dist_cen'] = np.select(conditions,change_with,df_cen['dist_cen'])


#### Create unique IDs in census 

Add a village counter 

In [9]:
# Creating a variable for count of villages for a given ID. Since the ID is made using district+block+gp 
#and a gp has jurisdiction over multiple villages.
df_cen['id_cen'] = df_cen.dist_cen + "_" + df_cen.blk_cen + "_" + df_cen.gp_cen
df_cen['n_vill'] = 1

#### groupby census villages to GP level 


In [10]:
cen_gp_lvl = df_cen.groupby(['id_cen']).sum().reset_index()
cen_gp_lvl[['id_cen', 'Total Population of Village', 'n_vill' ]].head()

C:\Users\Lahiri\AppData\Local\Temp\ipykernel_51020\955818899.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cen_gp_lvl = df_cen.groupby(['id_cen']).sum().reset_index()


,id_cen,Total Population of Village,n_vill
0,anugul_anugul_angar bandha,4484,3
1,anugul_anugul_antulia,5648,10
2,anugul_anugul_badakantakul,6334,10
3,anugul_anugul_badakera,7259,8
4,anugul_anugul_balanga,2643,7


## Generating variables


### Demographics

- Share of SC, ST population

- total geographical area

- total population of village

- male population

- female population

- SC population

- ST population

- Nearest town distance from village


---------------------------------





In [11]:
#final list of columns we are keeping from census 
cen_final_list = ['id_cen', 
'Total Geographical Area (in Hectares)', 
'Forest Area (in Hectares)',
'Total   Households ',
'Total Population of Village',
'Total Male Population of Village',
'Total Female Population of Village',
'Total Scheduled Castes Population of Village',
'Total Scheduled Tribes Population of Village', ]
cen_final_list = cen_final_list + health_cols + educ_cols + fin_cols + road_cols
cen_fnl = cen_gp_lvl[cen_final_list]

cen_fnl.head()
cen_fnl.shape
cen_fnl.columns

,id_cen,Total Geographical Area (in Hectares),Forest Area (in Hectares),Total Households,Total Population of Village,Total Male Population of Village,Total Female Population of Village,Total Scheduled Castes Population of Village,Total Scheduled Tribes Population of Village,Community Health Centre (Numbers),...,Cooperative Bank (Status A(1)/NA(2)),Agricultural Credit Societies (Status A(1)/NA(2)),Self - Help Group (SHG) (Status A(1)/NA(2)),National Highway (Status A(1)/NA(2)),State Highway (Status A(1)/NA(2)),Major District Road (Status A(1)/NA(2)),Other District Road (Status A(1)/NA(2)),Black Topped (pucca) Road (Status A(1)/NA(2)),Gravel (kuchha) Roads (Status A(1)/NA(2)),All Weather Road (Status A(1)/NA(2))
0,anugul_anugul_angar bandha,728.0,4.52,937.0,4484,2321.0,2163.0,461.0,152.0,0.0,...,0.0,0.0,3.0,0.0,0.0,1.0,3.0,2.0,3.0,0.0
1,anugul_anugul_antulia,2620.0,855.44,1233.0,5648,2844.0,2804.0,1080.0,1940.0,0.0,...,0.0,0.0,9.0,0.0,0.0,1.0,0.0,6.0,9.0,1.0
2,anugul_anugul_badakantakul,3122.0,811.65,1509.0,6334,3235.0,3099.0,1125.0,1245.0,0.0,...,0.0,0.0,5.0,0.0,0.0,3.0,3.0,7.0,8.0,8.0
3,anugul_anugul_badakera,1533.0,130.24,1583.0,7259,3733.0,3526.0,2367.0,162.0,0.0,...,0.0,0.0,8.0,1.0,1.0,1.0,5.0,8.0,8.0,8.0
4,anugul_anugul_balanga,1358.0,236.76,614.0,2643,1304.0,1339.0,358.0,686.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,1.0,6.0,3.0,2.0


(6241, 113)

Index(['id_cen', 'Total Geographical Area (in Hectares)',
       'Forest Area (in Hectares)', 'Total   Households ',
       'Total Population of Village', 'Total Male Population of Village',
       'Total Female Population of Village',
       'Total Scheduled Castes Population of Village',
       'Total Scheduled Tribes Population of Village',
       'Community Health Centre (Numbers)',
       ...
       'Cooperative Bank (Status A(1)/NA(2))',
       'Agricultural Credit Societies (Status A(1)/NA(2))',
       'Self - Help Group (SHG) (Status A(1)/NA(2))',
       'National Highway (Status A(1)/NA(2))',
       'State Highway (Status A(1)/NA(2))',
       'Major District Road (Status A(1)/NA(2))',
       'Other District Road (Status A(1)/NA(2))',
       'Black Topped (pucca) Road (Status A(1)/NA(2))',
       'Gravel (kuchha) Roads (Status A(1)/NA(2))',
       'All Weather Road (Status A(1)/NA(2))'],
      dtype='object', length=113)

In [12]:
#Forest area as percentage of total geographical area 
cen_fnl['forest_perc'] = cen_fnl[ 'Forest Area (in Hectares)']/(cen_fnl['Total Geographical Area (in Hectares)'] + 1)

#Scheduled castes as percentage of total pop
cen_fnl['sc_perc'] = cen_fnl['Total Scheduled Castes Population of Village']/(cen_fnl['Total Population of Village'] + 1) 

#Scheduled Tribes 
cen_fnl['st_perc'] = cen_fnl['Total Scheduled Tribes Population of Village']/(cen_fnl['Total Population of Village'] + 1) 

#Sex ratio 
cen_fnl['fem_to_male'] = cen_fnl['Total Female Population of Village']/(cen_fnl['Total Male Population of Village'] + 1)

C:\Users\Lahiri\AppData\Local\Temp\ipykernel_51020\867082080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cen_fnl['forest_perc'] = cen_fnl[ 'Forest Area (in Hectares)']/(cen_fnl['Total Geographical Area (in Hectares)'] + 1)
C:\Users\Lahiri\AppData\Local\Temp\ipykernel_51020\867082080.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cen_fnl['sc_perc'] = cen_fnl['Total Scheduled Castes Population of Village']/(cen_fnl['Total Population of Village'] + 1)
C:\Users\Lahiri\AppData\Local\Temp\ipykernel_5

# Cleaning expenditure data


In [13]:
exp = pd.read_csv("../data/exp_t4.csv")

#### Creating uniue IDs for expenditure data 


In [14]:
#Cleaning names
exp['blk_exp'] = exp['Block_Panchayat'].str.lower().str.strip()
exp['dist_exp']=exp['District_Panchayat'].str.lower().str.strip()
exp['gp_exp'] = exp['Village_Panchayat'].str.lower().str.strip()

#Creating expenditure ID
exp['id_exp'] = exp['dist_exp'] + "_" + exp['blk_exp'] + "_" + exp['gp_exp']

#### Group to sector level

We groupby expenditure level data by id, sector of activity and year 


In [15]:
exp['own_fund_exp']=np.where(exp.scheme_name=="Own Funds", exp['estimated_cost'], 0)

In [16]:

exp['n_activity'] = 1

exp = exp.groupby(['id_exp', 'sector', 'Plan_Year']).agg(tot_spend=('estimated_cost', np.sum),
                                               n_act = ('n_activity', np.sum),
                                               sc_fund = ('sc_fund', np.sum),
                                               st_fund = ('st_fund', np.sum),
                                               own_spend = ('own_fund_exp', np.sum)).reset_index()
                                               
                                               
exp.head()

,id_exp,sector,Plan_Year,tot_spend,n_act,sc_fund,st_fund,own_spend
0,anugul_anugul_angarbandha,Administrative & Technical Support,2015-2016,71000.0,5,0,0,0.0
1,anugul_anugul_angarbandha,Administrative & Technical Support,2016-2017,94000.0,6,0,0,0.0
2,anugul_anugul_angarbandha,Administrative & Technical Support,2017-2018,313600.0,7,0,0,0.0
3,anugul_anugul_angarbandha,Administrative & Technical Support,2018-2019,843600.0,9,0,0,0.0
4,anugul_anugul_angarbandha,Administrative & Technical Support,2019-2020,772629.0,5,0,0,0.0


## Creating district, block and gp level names from id

Since we lost them in grouping earlier 


In [17]:
#From id_cen, splitting to create dist, block and gp names 
cen_fnl[['dist_cen', 'blk_cen','gp_cen']] = cen_fnl['id_cen'].str.split('_', expand=True)

# some village names have spaces and dots, removing that
cen_fnl['id_cen'] = cen_fnl['id_cen'].str.replace(" ", "").str.replace(".", "") 

#From id_cen, splitting to create dist, block and gp names 
exp[['dist_exp', 'blk_exp','gp_exp']] = exp['id_exp'].str.split('_', expand=True)

# some village names have spaces and dots, removing that
exp['id_exp'] = exp['id_exp'].str.replace(" ", "").str.replace(".", "")

C:\Users\Lahiri\AppData\Local\Temp\ipykernel_51020\2824929699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cen_fnl[['dist_cen', 'blk_cen','gp_cen']] = cen_fnl['id_cen'].str.split('_', expand=True)
C:\Users\Lahiri\AppData\Local\Temp\ipykernel_51020\2824929699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cen_fnl[['dist_cen', 'blk_cen','gp_cen']] = cen_fnl['id_cen'].str.split('_', expand=True)
C:\Users\Lahiri\AppData\Local\Temp\ipykernel_51020\2824929699.py:2: SettingWithCopyWarning: 
A value is 

In [18]:
# exp['id_exp'][exp['id_exp'].str.contains("\(")]
# rayagada(chandili)
cen_fnl[cen_fnl['gp_cen'].str.contains("rayagada")]
cen_fnl[cen_fnl['id_cen'].str.contains("chandili")]
exp[exp['gp_exp'].str.contains("rayagada")]
exp[exp['id_exp'].str.contains("chandili")]

,id_cen,Total Geographical Area (in Hectares),Forest Area (in Hectares),Total Households,Total Population of Village,Total Male Population of Village,Total Female Population of Village,Total Scheduled Castes Population of Village,Total Scheduled Tribes Population of Village,Community Health Centre (Numbers),...,Black Topped (pucca) Road (Status A(1)/NA(2)),Gravel (kuchha) Roads (Status A(1)/NA(2)),All Weather Road (Status A(1)/NA(2)),forest_perc,sc_perc,st_perc,fem_to_male,dist_cen,blk_cen,gp_cen
2015,gajapati_rayagada_rayagada,2196.0,220.27,222.0,976,453.0,523.0,0.0,969.0,0.0,...,1.0,6.0,2.0,0.100259,0.0,0.991812,1.151982,gajapati,rayagada,rayagada


,id_cen,Total Geographical Area (in Hectares),Forest Area (in Hectares),Total Households,Total Population of Village,Total Male Population of Village,Total Female Population of Village,Total Scheduled Castes Population of Village,Total Scheduled Tribes Population of Village,Community Health Centre (Numbers),...,Black Topped (pucca) Road (Status A(1)/NA(2)),Gravel (kuchha) Roads (Status A(1)/NA(2)),All Weather Road (Status A(1)/NA(2)),forest_perc,sc_perc,st_perc,fem_to_male,dist_cen,blk_cen,gp_cen
4268,koraput_kotpad_chandili,2347.0,61.06,1736.0,6929,3446.0,3483.0,1448.0,3881.0,0.0,...,6.0,9.0,0.0,0.026005,0.208947,0.560029,1.010444,koraput,kotpad,chandili


,id_exp,sector,Plan_Year,tot_spend,n_act,sc_fund,st_fund,own_spend,dist_exp,blk_exp,gp_exp
111087,gajapati_rayagada_rayagada,Administrative & Technical Support,2015-2016,194435.0,4,0,0,0.0,gajapati,rayagada,rayagada
111088,gajapati_rayagada_rayagada,Administrative & Technical Support,2017-2018,252552.0,5,0,0,0.0,gajapati,rayagada,rayagada
111089,gajapati_rayagada_rayagada,Administrative & Technical Support,2018-2019,57000.0,1,0,0,0.0,gajapati,rayagada,rayagada
111090,gajapati_rayagada_rayagada,Administrative & Technical Support,2019-2020,362000.0,4,0,0,0.0,gajapati,rayagada,rayagada
111091,gajapati_rayagada_rayagada,Administrative & Technical Support,2020-2021,417643.0,4,0,0,0.0,gajapati,rayagada,rayagada
111092,gajapati_rayagada_rayagada,Administrative & Technical Support,2021-2022,567717.0,5,0,0,74.0,gajapati,rayagada,rayagada
111093,gajapati_rayagada_rayagada,Administrative & Technical Support,2022-2023,517540.0,1,0,0,0.0,gajapati,rayagada,rayagada
111094,gajapati_rayagada_rayagada,Cultural activities,2020-2021,200000.0,1,0,0,0.0,gajapati,rayagada,rayagada
111095,gajapati_rayagada_rayagada,Drinking water,2015-2016,183831.0,2,0,0,0.0,gajapati,rayagada,rayagada
111096,gajapati_rayagada_rayagada,Drinking water,2017-2018,200000.0,2,0,0,0.0,gajapati,rayagada,rayagada


,id_exp,sector,Plan_Year,tot_spend,n_act,sc_fund,st_fund,own_spend,dist_exp,blk_exp,gp_exp
227655,koraput_kotpad_chandili,Administrative & Technical Support,2016-2017,197760.0,10,0,0,0.0,koraput,kotpad,chandili
227656,koraput_kotpad_chandili,Administrative & Technical Support,2017-2018,304760.0,11,0,0,0.0,koraput,kotpad,chandili
227657,koraput_kotpad_chandili,Administrative & Technical Support,2018-2019,527400.0,6,0,0,0.0,koraput,kotpad,chandili
227658,koraput_kotpad_chandili,Administrative & Technical Support,2019-2020,90000.0,2,0,0,0.0,koraput,kotpad,chandili
227659,koraput_kotpad_chandili,Administrative & Technical Support,2020-2021,48000.0,2,0,0,0.0,koraput,kotpad,chandili
...,...,...,...,...,...,...,...,...,...,...,...
305430,rayagada_rayagada_chandili,Sanitation,2021-2022,2806124.0,24,0,0,0.0,rayagada,rayagada,chandili
305431,rayagada_rayagada_chandili,Sanitation,2022-2023,1522142.0,10,0,0,0.0,rayagada,rayagada,chandili
305432,rayagada_rayagada_chandili,Social welfare,2016-2017,1400000.0,4,0,0,0.0,rayagada,rayagada,chandili
305433,rayagada_rayagada_chandili,Social welfare,2017-2018,1790000.0,7,0,0,0.0,rayagada,rayagada,chandili


#### Processing for fuzzy matching 

In [19]:
#perfect matches of IDs between datasets
perfect_matches = cen_fnl['id_cen'][cen_fnl['id_cen'].isin(exp['id_exp'])]

#removing perfect matches from list of unique IDs in census data
imperfect_cen = list(set(cen_fnl['id_cen'].unique().tolist()) - set(perfect_matches.unique().tolist()))

#removing perfect matches from list of unique IDs in expenditure data
imperfect_exp = list(set(exp['id_exp'].unique().tolist()) - set(perfect_matches.unique().tolist()))

In [20]:
print("Number of unique IDs in expenditure data: " + str(len(exp['id_exp'].unique())))
print("Number of unique IDs in census data: " + str(len(cen_fnl['id_cen'].unique())))
print("Number of unique IDs that perfectly match already between two dataframes: " + str(len(perfect_matches)))


Number of unique IDs in expenditure data: 6903
Number of unique IDs in census data: 6241
Number of unique IDs that perfectly match already between two dataframes: 2997


## 2. Fuzzy merging 


2997 IDs match perfectly. There are 3906 non-matching IDs in expenditure data, and 3244 non-matching IDs in census data. 

Some GPs were newly created after the Census (from 2012 - 2021), so some GPs will not exist in the Census data.

However, due to lack of standardized spellings, there are ID strings that should match (i.e. they refer to the same entity), but do not.

Here, we use approximate string matching (or fuzzy matching). There are multiple libraries, but we use `fuzzywuzzy`.

**FuzzyWuzzy** uses Levenshtein Distance to calculate distance between patterns. There are four metrics to measure the similarity/difference between strings.

**Fuzz Ratio** - Calculate distance between ordering of tokens. We use this.

**Partial Ratio** - Take the shortest string, match with all substring of longer string

**token_sort_ratio** - Removes punctutation, sorts string alphabetically. 

**token_set_ratio** - Takes out common tokens instead of simply tokenizing. Extra words are removed.

A single ID string in the EXP data will match with multiple IDs in CEN. 
Each match pair will have its own score, indicating how similar the strings are. 

Perfect matches would get a score of 100, but we have already removed them. 

We initially keep only matches with scores > 80. This will still yield multiple candidate matches for each ID, with different scores. We only keep the match pair with the highest score.


After some manual reviewing, we find that most of the match pairs between the score of 80 - 90 are spurious.  We hence drop matches below the score of 90. (**Note: The last part is a trial and error process**)

Around 192 GPs could not even find a string similar with a score of 80 - indicates these are new names/ creations. We drop them.

In [21]:
metric = fuzz.ratio
#metric = fuzz.partial_ratio
#metric = fuzz.token_sort_ratio
#metric = fuzz.token_set_ratio

thresh = 80

ca = np.array(imperfect_exp) #array of unique IDs in expenditure data that did not have a perfect match as found above
cb = np.array(imperfect_cen) #array of unique IDs in census data that did not have a perfect match as found above



#Parallel processing for faster results
def parallel_fuzzy_match(idxa,idxb):
    return [ca[idxa],cb[idxb],metric(ca[idxa],cb[idxb])] 
results = Parallel(n_jobs=-1,verbose=1)(delayed(parallel_fuzzy_match)(idx1, idx2) for idx1 in range(len(ca)) for idx2 in range(len(cb)) \
                   if(metric(ca[idx1],cb[idx2]) > thresh))
scores = pd.DataFrame(results,columns = ["id_exp","id_cen","Score"])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 240 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1192 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 2592 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 4392 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 6592 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 9192 tasks      | elapsed:   33.3s
[Parallel(n_jobs=-1)]: Done 12192 tasks      | elapsed:   44.2s
[Parallel(n_jobs=-1)]: Done 15592 tasks      | elapsed:   56.3s
[Parallel(n_jobs=-1)]: Done 16554 out of 16554 | elapsed:   59.1s finished


#### Cleaning Fuzzy scores tables

In [22]:
df_scores = scores 

#IDs in expenditure had multiple matches as long as match score was >80, therefore keeping only matchesd with best score
best_matches = df_scores.sort_values('Score', ascending = False).drop_duplicates('id_exp', keep='first') 
best_matches.reset_index().drop(columns = 'index', inplace = True)

#appending perfect matches to the above list. In this dataframe, both columns have same IDs, since they were perfect matches 
#from the beginning
perfect_match = pd.DataFrame({'id_cen':perfect_matches.unique().tolist(),
                              'id_exp':perfect_matches.unique().tolist()})
total_matches = best_matches.append(perfect_match).reset_index()
total_matches.drop(columns = 'index', inplace = True)

#filling score value for perfect matches as 100. 
total_matches = total_matches.fillna(100)

best_matches #contains only fuzzed matched IDs
best_matches.shape
total_matches # after appending perfect matches best matches 
total_matches.shape

#This shows that between 80 and 87 fuzz match is getting it wrong quite often, but it does catch some great cases. 


C:\Users\Lahiri\AppData\Local\Temp\ipykernel_51020\368570965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_matches = best_matches.append(perfect_match).reset_index()


,id_exp,id_cen,Score
1908,cuttack_narasinghpur_paikabarbati,cuttack_narasinghpur_paikabarabati,99
534,jagatsinghapur_jagatsinghpur_jankoti,jagatsinghapur_jagatsinghapur_jankoti,99
14492,kendujhar_harichadanpur_pithagola,kendujhar_harichandanpur_pithagola,99
15917,sonepur_birmaharajpur_kenjhariapali,sonepur_biramaharajpur_kenjhariapali,99
3893,nabarangpur_tentulikhunti_anchalagumma,nabarangpur_tentulikhunti_anchalaguma,99
...,...,...,...
12664,khordha_jatni_tirimal,khordha_jatani_kantia,81
990,malkangiri_korukonda_nakamamudi,malkangiri_korkunda_kapatuti,81
12360,kendujhar_joda_serenda,kendujhar_joda_balda,81
12335,puri_krushnaprasad_gopinathapur,puri_krushnaprasad_parijipur,81


(3714, 3)

,id_exp,id_cen,Score
0,cuttack_narasinghpur_paikabarbati,cuttack_narasinghpur_paikabarabati,99.0
1,jagatsinghapur_jagatsinghpur_jankoti,jagatsinghapur_jagatsinghapur_jankoti,99.0
2,kendujhar_harichadanpur_pithagola,kendujhar_harichandanpur_pithagola,99.0
3,sonepur_birmaharajpur_kenjhariapali,sonepur_biramaharajpur_kenjhariapali,99.0
4,nabarangpur_tentulikhunti_anchalagumma,nabarangpur_tentulikhunti_anchalaguma,99.0
...,...,...,...
6706,sundargarh_sundargarh_lankahuda,sundargarh_sundargarh_lankahuda,100.0
6707,sundargarh_sundargarh_ledhimang,sundargarh_sundargarh_ledhimang,100.0
6708,sundargarh_sundargarh_majhapara,sundargarh_sundargarh_majhapara,100.0
6709,sundargarh_sundargarh_masnikani,sundargarh_sundargarh_masnikani,100.0


(6711, 3)

In [23]:
print(str(len(exp['id_exp'].unique())-len(total_matches['id_exp'].unique()))+ 
      " IDs in the expenditure dataset whose match score with IDs in the census was less than 80, i.e. these IDs did \
 not get matched")




192 IDs in the expenditure dataset whose match score with IDs in the census was less than 80, i.e. these IDs did  not get matched


In [24]:
random.seed(27112025)

#example of why we subset for scores above 93 
#the first row should have been fuzzy matched and illustrates some data loss we face 
#due to subsetting 

total_matches[(total_matches.Score <= 89)].sample(3)

# Subset to scores >93
total_matches[(total_matches.Score >= 93)]

,id_exp,id_cen,Score
3699,balangir_patnagarh_kerbeda,balangir_patnagarh_solbandha,81.0
3627,kalahandi_jayapatna_badatemari,kalahandi_jayapatna_ranmal,82.0
3253,rayagada_bissamcuttack_kutraguda,rayagada_bishamakatak_kutraguda,86.0


,id_exp,id_cen,Score
0,cuttack_narasinghpur_paikabarbati,cuttack_narasinghpur_paikabarabati,99.0
1,jagatsinghapur_jagatsinghpur_jankoti,jagatsinghapur_jagatsinghapur_jankoti,99.0
2,kendujhar_harichadanpur_pithagola,kendujhar_harichandanpur_pithagola,99.0
3,sonepur_birmaharajpur_kenjhariapali,sonepur_biramaharajpur_kenjhariapali,99.0
4,nabarangpur_tentulikhunti_anchalagumma,nabarangpur_tentulikhunti_anchalaguma,99.0
...,...,...,...
6706,sundargarh_sundargarh_lankahuda,sundargarh_sundargarh_lankahuda,100.0
6707,sundargarh_sundargarh_ledhimang,sundargarh_sundargarh_ledhimang,100.0
6708,sundargarh_sundargarh_majhapara,sundargarh_sundargarh_majhapara,100.0
6709,sundargarh_sundargarh_masnikani,sundargarh_sundargarh_masnikani,100.0


## Merge Census + Expenditure data

In [25]:
#merging matched dataset with expenditure 
exp_matches = pd.merge(exp, total_matches, how = 'inner', left_on = 'id_exp', right_on='id_exp')
cen_fnl2= cen_fnl.drop(columns = ['dist_cen','blk_cen','gp_cen'])

#merging matched expenditure dataset with census data 
cen_exp = pd.merge(cen_fnl, exp_matches, how = 'inner', left_on = 'id_cen', right_on='id_cen')

In [26]:
cen_exp.head()
print("Shape of final merged data: " + str(cen_exp.shape))
print("Shape of original expenditure data: " + str(exp.shape) + " (Because this is the bigger dataset, so benchmarking against this one)")

,id_cen,Total Geographical Area (in Hectares),Forest Area (in Hectares),Total Households,Total Population of Village,Total Male Population of Village,Total Female Population of Village,Total Scheduled Castes Population of Village,Total Scheduled Tribes Population of Village,Community Health Centre (Numbers),...,Plan_Year,tot_spend,n_act,sc_fund,st_fund,own_spend,dist_exp,blk_exp,gp_exp,Score
0,anugul_anugul_angarbandha,728.0,4.52,937.0,4484,2321.0,2163.0,461.0,152.0,0.0,...,2015-2016,71000.0,5,0,0,0.0,anugul,anugul,angarbandha,100.0
1,anugul_anugul_angarbandha,728.0,4.52,937.0,4484,2321.0,2163.0,461.0,152.0,0.0,...,2016-2017,94000.0,6,0,0,0.0,anugul,anugul,angarbandha,100.0
2,anugul_anugul_angarbandha,728.0,4.52,937.0,4484,2321.0,2163.0,461.0,152.0,0.0,...,2017-2018,313600.0,7,0,0,0.0,anugul,anugul,angarbandha,100.0
3,anugul_anugul_angarbandha,728.0,4.52,937.0,4484,2321.0,2163.0,461.0,152.0,0.0,...,2018-2019,843600.0,9,0,0,0.0,anugul,anugul,angarbandha,100.0
4,anugul_anugul_angarbandha,728.0,4.52,937.0,4484,2321.0,2163.0,461.0,152.0,0.0,...,2019-2020,772629.0,5,0,0,0.0,anugul,anugul,angarbandha,100.0


Shape of final merged data: (325225, 132)
Shape of original expenditure data: (332778, 11) (Because this is the bigger dataset, so benchmarking against this one)


In [27]:
#Without fuzzy merging, dataset size halves

merged_normal = pd.merge(exp,cen_fnl,right_on = 'id_cen', left_on = 'id_exp', how = 'inner', validate = "many_to_one")

merged_normal.shape


(149739, 131)

In [30]:
# Exporting final dataframe which will be used for analysis 
# uncomment below line to do so 
cen_exp.to_csv("../data/analysis_df.csv") 

## Shapefile data fuzzy matching 

In [43]:
od_gp= gpd.read_file(r"C:\Users\Lahiri\Dropbox\ppol564\od_gp_ntl_2011_2018.shp\od_gp_ntl_2011_2018.shp.shp")

In [44]:
od_gp.drop(columns = ['ntl_2011',
 'ntl_2012',
 'ntl_2013',
 'ntl_2014',
 'ntl_2015',
 'ntl_2016',
 'ntl_2017',
 'ntl_2018'], inplace = True)


In [45]:
od_gp['ntl_avg'] = od_gp[['ntl_mean_5','ntl_mean_6','ntl_mean_7','ntl_mean_8']].mean(axis = 1)
od_gp

,OBJECTID,S_CODE,S_NAME,D_CODE,D_NAME,B_CODE,B_NAME,GP_CODE,GP_NAME,NO_HH,...,ntl_mean_2,ntl_mean_1,ntl_mean_3,ntl_mean_4,ntl_mean_5,ntl_mean_6,ntl_mean_7,ntl_mean_8,geometry,ntl_avg
0,1.0,21,ODISHA,384,ANUGUL,3276.0,ANUGUL,115550.0,ANGARBANDHA,937.0,...,8.352941,7.894737,9.000000,12.473684,15.315789,13.473684,15.000000,15.052632,"POLYGON ((85.14450 20.79444, 85.14540 20.79372...",14.710526
1,2.0,21,ODISHA,384,ANUGUL,3276.0,ANUGUL,115551.0,ANTULA,1233.0,...,0.000000,0.296875,0.000000,6.203125,6.984375,5.859375,7.000000,7.375000,"MULTIPOLYGON (((84.82732 20.80466, 84.82849 20...",6.804688
2,3.0,21,ODISHA,384,ANUGUL,3276.0,ANUGUL,115552.0,BADAKANTAKUL,1509.0,...,0.567164,0.571429,0.642857,6.500000,6.657143,6.114286,6.885714,7.228571,"MULTIPOLYGON (((85.11752 20.67779, 85.11805 20...",6.721429
3,4.0,21,ODISHA,384,ANUGUL,3276.0,ANUGUL,115553.0,BADAKERA,1583.0,...,6.250000,7.125000,7.450000,9.125000,10.550000,9.400000,10.675000,11.150000,"POLYGON ((85.02422 20.81624, 85.02475 20.81605...",10.443750
4,5.0,21,ODISHA,384,ANUGUL,3276.0,ANUGUL,115554.0,BALANGA,614.0,...,0.000000,0.132075,0.000000,6.000000,6.207547,6.094340,6.905660,6.867925,"MULTIPOLYGON (((84.99345 20.69073, 84.99375 20...",6.518868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6227,6228.0,21,ODISHA,385,NAYAGARH,3526.0,KHANDAPARA,120642.0,KANTILO,2118.0,...,3.666667,3.482759,5.034483,7.068966,7.379310,7.000000,8.689655,8.862069,"POLYGON ((85.18452 20.37648, 85.18454 20.37663...",7.982759
6228,6229.0,21,ODISHA,386,KHORDHA,3456.0,BALIANTA,119544.0,BALIANTA(PRATAPSASAN),2576.0,...,19.263158,14.928571,18.642857,24.142857,26.142857,25.285714,24.357143,26.357143,"POLYGON ((85.86644 20.22044, 85.86666 20.22028...",25.535714
6229,6230.0,21,ODISHA,388,GANJAM,3382.0,RANGEILUNDA,117956.0,GOLABANDHA,1392.0,...,19.000000,13.846154,14.461538,16.076923,17.076923,17.153846,20.230769,21.153846,"POLYGON ((84.87267 19.25958, 84.87292 19.25958...",18.903846
6230,6231.0,21,ODISHA,372,SAMBALPUR,3566.0,RENGALI,121420.0,RENGALI,2529.0,...,13.357143,10.333333,11.766667,18.033333,21.666667,19.933333,18.900000,19.966667,"POLYGON ((84.03449 21.65623, 84.03474 21.65635...",20.116667


### Creating IDs likes as before

In [46]:
od_gp['D_NAME'] = od_gp['D_NAME'].str.lower().str.strip().str.replace(" ","")
od_gp['B_NAME'] = od_gp['B_NAME'].str.lower().str.strip().str.replace(" ","")
od_gp['GP_NAME'] = od_gp['GP_NAME'].str.lower().str.strip().str.replace(" ","")

### Renaming districts like we did for census

In [47]:
conditions = [od_gp.D_NAME == 'baudh',od_gp.D_NAME == 'debagarh',od_gp.D_NAME == 'nabarangapur',od_gp.D_NAME == 'subarnapur']
change_with = ['boudh', 'deogarh', 'nabarangpur', 'sonepur']
od_gp['D_NAME'] = np.select(conditions,change_with,od_gp['D_NAME'])
od_gp['id'] = od_gp['D_NAME'] +'_'+od_gp['B_NAME']+ '_'+od_gp['GP_NAME']


### Removing village names with brackets, since their case is uncleaer

In [48]:
od_gp = od_gp[~od_gp['id'].str.contains(r'\(')]

### Preparing for fuzzy match. Same process as earlier

In [49]:
## Perfect matches between od_gp and expenditure

perfect_matches_2 = exp['id_exp'][exp['id_exp'].isin(od_gp['id'])]

In [50]:
print('Number of perfect matches between satellite data and expenditure data:%.3f'%len(perfect_matches_2.unique()))

Number of perfect matches between satellite data and expenditure data:5387.000


In [51]:
imperfect_exp_od = list(set(exp['id_exp'].unique().tolist()) - set(perfect_matches_2.unique().tolist()))
imperfect_od = list(set(od_gp['id'].unique().tolist()) - set(perfect_matches_2.unique().tolist()))


In [52]:
len(imperfect_exp_od)
len(imperfect_od)

1516

827

In [53]:
metric = fuzz.ratio
#metric = fuzz.partial_ratio
#metric = fuzz.token_sort_ratio
#metric = fuzz.token_set_ratio

thresh = 80

ca = np.array(imperfect_exp_od) #array of unique IDs in expenditure data that did not have a perfect match as found above
cb = np.array(imperfect_od) #array of unique IDs in census data that did not have a perfect match as found above



#Parallel processing for faster results
def parallel_fuzzy_match(idxa,idxb):
    return [ca[idxa],cb[idxb],metric(ca[idxa],cb[idxb])] 
results = Parallel(n_jobs=-1,verbose=1)(delayed(parallel_fuzzy_match)(idx1, idx2) for idx1 in range(len(ca)) for idx2 in range(len(cb)) \
                   if(metric(ca[idx1],cb[idx2]) > thresh))
scores = pd.DataFrame(results,columns = ["id_exp","id_od_gp","Score"])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 1248 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 6634 out of 6634 | elapsed:    4.4s finished


In [55]:
df_scores = scores 

#IDs in expenditure had multiple matches as long as match score was >80, therefore keeping only matchesd with best score
best_matches = df_scores.sort_values('Score', ascending = False).drop_duplicates('id_exp', keep='first') 

#between 87 and 90, separately checked for matches for each score and noted the index of the matches that were not good.
#dropping them

#Overall keeping scores above 86
best_matches = best_matches[best_matches.Score > 88]
best_matches.reset_index().drop(columns = 'index', inplace = True)

In [56]:
best_matches

,id_exp,id_od_gp,Score
1910,jagatsinghapur_jagatsinghpur_khairanga,jagatsinghpur_jagatsinghpur_khairanga,99
3517,jagatsinghapur_raghunathpur_tarapur,jagatsinghpur_raghunathpur_tarapur,99
1221,jagatsinghapur_raghunathpur_redhua,jagatsinghpur_raghunathpur_redhua,99
1437,jagatsinghapur_tirtol_jagannathpur,jagatsinghpur_tirtol_jagannathpur,99
4483,jagatsinghapur_raghunathpur_purunabasanta,jagatsinghpur_raghunathpur_purunabasanta,99
...,...,...,...
1702,baleshwar_soro_kesharipur,baleswar_soro_kedarpur,89
4474,cuttack_nischintakoili_badakhira,cuttack_nischintakoil_bandhakatia,89
1502,gajapati_rayagada_ameda,gajapati_rayagada_dambal,89
29,ganjam_buguda_biranchipurkaradabadi,ganjam_buguda_biranchipurkar,89


In [57]:
#appending the perfect matches 
perfect_match = pd.DataFrame({'id_exp':perfect_matches_2.unique().tolist(),
                              'id_od_gp':perfect_matches_2.unique().tolist()})
total_matches = best_matches.append(perfect_match).reset_index()
total_matches.drop(columns = 'index', inplace = True)

#filling score value for perfect matches as 100. 
total_matches = total_matches.fillna(100)

C:\Users\Lahiri\AppData\Local\Temp\ipykernel_51020\4178317683.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_matches = best_matches.append(perfect_match).reset_index()


In [58]:
total_matches

,id_exp,id_od_gp,Score
0,jagatsinghapur_jagatsinghpur_khairanga,jagatsinghpur_jagatsinghpur_khairanga,99.0
1,jagatsinghapur_raghunathpur_tarapur,jagatsinghpur_raghunathpur_tarapur,99.0
2,jagatsinghapur_raghunathpur_redhua,jagatsinghpur_raghunathpur_redhua,99.0
3,jagatsinghapur_tirtol_jagannathpur,jagatsinghpur_tirtol_jagannathpur,99.0
4,jagatsinghapur_raghunathpur_purunabasanta,jagatsinghpur_raghunathpur_purunabasanta,99.0
...,...,...,...
6170,sundargarh_tangarpali_ratanpur,sundargarh_tangarpali_ratanpur,100.0
6171,sundargarh_tangarpali_remenda,sundargarh_tangarpali_remenda,100.0
6172,sundargarh_tangarpali_tangarpali,sundargarh_tangarpali_tangarpali,100.0
6173,sundargarh_tangarpali_tasladihi,sundargarh_tangarpali_tasladihi,100.0


### Reshaping exp data 

In [59]:
#renaming health and sanitation to H&S since there is separate sector called health 
exp['sector'] = np.where(exp.sector == 'Health & Sanitation','H&S',exp['sector'])

In [60]:
#keeping only pertinent variables from expenditure data
exp2 = exp[['id_exp','sector','Plan_Year','tot_spend']]

#Creating average spending in each sector over 2015 to 2022.
exp_wide = exp2.pivot_wider(
    index = 'id_exp', 
    names_from = ['sector','Plan_Year'],
    values_from = ['tot_spend'],
    names_glue = "{sector}_{Plan_Year}"
  )

# check = exp2.groupby(['id_exp','sector']).mean()
exp_wide.fillna(0, inplace = True)
for i in exp2.sector.unique().tolist():
    a = [col for col in exp_wide.columns.tolist() if i in col ]
    exp_wide[i+'_avg'] = exp_wide[[col for col in exp_wide.columns.tolist() if i in col ]].mean(axis = 1)
    exp_wide.drop(columns = a, inplace = True )

In [61]:
#merging matched dataset with wide expenditure data
shp_matches = pd.merge(exp_wide, total_matches, how = 'inner', left_on = 'id_exp', right_on='id_exp')

#merging matched expenditure dataset with odgp data 
level2_match = od_gp.merge(shp_matches, how = 'inner', left_on = 'id', right_on='id_od_gp')

level2_match = level2_match[[
 'id_exp',
 'Administrative & Technical Support_avg',
 'Cultural activities_avg',
 'Drinking water_avg',
 'Education_avg',
 'GP Office Infrastructure_avg',
 'Health_avg',
 'Land improvement_avg',
 'Maintenance of community system_avg',
 'Markets and fairs_avg',
 'Roads_avg',
 'Sanitation_avg',
 'Social welfare_avg',
 'Technical training and vocational education_avg',
 'Women and child development_avg',
 'Rural electrification_avg',
 'Water Conservation_avg',
 'Family welfare_avg',
 'H&S_avg',
 'Minor forest produce_avg',
 'Public distribution system_avg',
 'Agriculture_avg',
 'Poverty allevation programme_avg',
 'Non-conventional energy sources_avg',
 'Tribal Welfare_avg',
 'Animal husbandry_avg',
 'Social forestry and farm forestry_avg',
 'Adult and non-formal education_avg',
 'Fisheries_avg',
 'Rural housing_avg',
 'Libraries_avg',
 'Welfare of the weaker sections_avg',
 'Small-scale industries_avg',
 'Fuel and fodder_avg',
 'Others_avg',
 'Khadi_avg',
 'ntl_avg']]

cen_exp_match = cen_exp[cen_exp.Plan_Year == '2015-2016']\
                        [['Total Population of Village','id_exp']].groupby('id_exp').mean().reset_index()
# merging matched od_gp+expenditure data with census+expenditure data 
final_match = level2_match.merge(cen_exp_match,
                       how = 'inner', 
                       left_on = ['id_exp'], 
                       right_on= ['id_exp'])

final_match = final_match[final_match['Total Population of Village']>0]
for i in final_match.columns.tolist():
    if 'avg' in i and 'ntl_avg' not in i:
        final_match[i] = final_match[i]/final_match['Total Population of Village']

In [62]:
final_match

,id_exp,Administrative & Technical Support_avg,Cultural activities_avg,Drinking water_avg,Education_avg,GP Office Infrastructure_avg,Health_avg,Land improvement_avg,Maintenance of community system_avg,Markets and fairs_avg,...,Fisheries_avg,Rural housing_avg,Libraries_avg,Welfare of the weaker sections_avg,Small-scale industries_avg,Fuel and fodder_avg,Others_avg,Khadi_avg,ntl_avg,Total Population of Village
0,anugul_anugul_angarbandha,63.097820,39.027654,180.199515,137.545495,22.692685,9.645406,6.969224,21.111786,47.390723,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.710526,4484.0
1,anugul_anugul_antula,47.027156,0.000000,127.592179,55.736345,6.639518,6.639518,4.426346,65.539173,27.664660,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.804688,5648.0
2,anugul_anugul_badakantakul,34.455320,0.000000,111.365587,48.251500,7.452400,3.453584,0.000000,32.904207,30.588885,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.721429,6334.0
3,anugul_anugul_badakera,32.246677,0.000000,94.867027,45.105869,23.489461,5.166001,0.861000,34.326388,54.243009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.443750,7259.0
4,anugul_anugul_balanga,52.721907,93.938706,233.655742,116.621027,33.106319,1.418842,12.296633,137.428112,94.589482,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.518868,2643.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6086,jagatsinghapur_erasama_nuagaon,25.493129,0.000000,159.071770,0.000000,98.842816,0.000000,7.232401,249.944118,23.505304,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.035714,5185.0
6087,sundargarh_balisankara_bandega,75.599778,0.000000,226.348251,37.462977,11.754906,0.000000,0.000000,53.152420,105.400777,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.755102,5402.0
6088,baleshwar_remuna_ganipur,3325.000000,0.000000,20683.804878,3125.000000,0.000000,364.000000,304.878049,46984.131098,457.530488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.375000,41.0
6089,nayagarh_khandapara_kantilo,55.609357,0.000000,201.207377,4.407906,37.822946,0.000000,24.602263,76.261850,172.985198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.982759,6097.0


In [63]:
# Exporting final dataframe which will be used for analysis 
# uncomment below line to do so 
final_match.to_csv("../data/averag_satellite.csv") 